In [1]:
import pandas as pd

# Charger le jeu de données
df_test = pd.read_parquet('data/green_tripdata_2021-03.parquet')

import mlflow
import pickle

# Spécifiez le nom du modèle enregistré
model_name = "nyc-taxi-regressor"

# Créez un client MLflow
client = mlflow.MlflowClient()

# Obtenez la dernière version du modèle en production
latest_production = client.get_latest_versions(model_name, stages=["Production"])[0]

# Téléchargez l'artefact 'dv.pkl'
artifact_path = client.download_artifacts(latest_production.run_id, "dv.pkl")

# Chargez le DictVectorizer
with open(artifact_path, 'rb') as f:
    dv = pickle.load(f)


# Sélectionnez les colonnes pertinentes
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

# Créez des dictionnaires pour le vecteur
dicts = df_test[categorical + numerical].to_dict(orient='records')

# Transformez les données
X_test = dv.transform(dicts)


from sklearn.metrics import mean_squared_error
import numpy as np

# Obtenez les versions des modèles
staging_model = client.get_latest_versions(model_name, stages=["Staging"])[0]
production_model = latest_production

# Chargez les modèles
model_staging = mlflow.pyfunc.load_model(f"models:/{model_name}/{staging_model.version}")
model_production = mlflow.pyfunc.load_model(f"models:/{model_name}/{production_model.version}")

# Effectuez les prédictions
y_pred_staging = model_staging.predict(X_test)
y_pred_production = model_production.predict(X_test)

# Calculez les RMSE
rmse_staging = mean_squared_error(df_test['fare_amount'], y_pred_staging, squared=False)
rmse_production = mean_squared_error(df_test['fare_amount'], y_pred_production, squared=False)

print(f"RMSE Staging: {rmse_staging:.2f}")
print(f"RMSE Production: {rmse_production:.2f}")

# Comparer les performances
if rmse_staging < rmse_production:
    # Promouvoir le modèle en Staging
    client.transition_model_version_stage(
        name=model_name,
        version=staging_model.version,
        stage="Production",
        archive_existing_versions=True
    )
    print(f"Le modèle version {staging_model.version} a été promu en Production.")
else:
    print("Le modèle en Production actuel est meilleur. Aucune mise à jour effectuée.")


FileNotFoundError: [Errno 2] No such file or directory: 'data/green_tripdata_2021-03.parquet'